In [9]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [10]:
""" 스키마 확인하기 """
df = spark.read.format("json").load("./data/flight-data/json/2015-summary.json") # 미국 교통통계국이 제공하는 항공운항 데이터
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



In [11]:
from pyspark.sql.functions import udf, col

# Use udf to define a row-at-a-time udf
@udf('long')
# Input/output are both a single double value
def plus_one(v):
      return v + 1

df.withColumn('v2', plus_one(col('count'))).show()

+--------------------+-------------------+-----+---+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count| v2|
+--------------------+-------------------+-----+---+
|       United States|            Romania|   15| 16|
|       United States|            Croatia|    1|  2|
|       United States|            Ireland|  344|345|
|               Egypt|      United States|   15| 16|
|       United States|              India|   62| 63|
|       United States|          Singapore|    1|  2|
|       United States|            Grenada|   62| 63|
|          Costa Rica|      United States|  588|589|
|             Senegal|      United States|   40| 41|
|             Moldova|      United States|    1|  2|
|       United States|       Sint Maarten|  325|326|
|       United States|   Marshall Islands|   39| 40|
|              Guyana|      United States|   64| 65|
|               Malta|      United States|    1|  2|
|            Anguilla|      United States|   41| 42|
|             Bolivia|      United States|   3

In [12]:
import pyarrow
pyarrow.__version__

'0.15.1'

In [5]:
from pyspark.sql.functions import pandas_udf, PandasUDFType

# Use pandas_udf to define a Pandas UDF
@pandas_udf('long', PandasUDFType.SCALAR)
# Input/output are both a pandas.Series of doubles

def pandas_plus_one(v):
    return v + 1

df.withColumn('v3', pandas_plus_one(col('count'))).show()

Py4JJavaError: An error occurred while calling o189.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 1 times, most recent failure: Lost task 0.0 in stage 3.0 (TID 4, localhost, executor driver): java.lang.IllegalArgumentException
	at java.nio.ByteBuffer.allocate(ByteBuffer.java:334)
	at org.apache.arrow.vector.ipc.message.MessageSerializer.readMessage(MessageSerializer.java:543)
	at org.apache.arrow.vector.ipc.message.MessageChannelReader.readNext(MessageChannelReader.java:58)
	at org.apache.arrow.vector.ipc.ArrowStreamReader.readSchema(ArrowStreamReader.java:132)
	at org.apache.arrow.vector.ipc.ArrowReader.initialize(ArrowReader.java:181)
	at org.apache.arrow.vector.ipc.ArrowReader.ensureInitialized(ArrowReader.java:172)
	at org.apache.arrow.vector.ipc.ArrowReader.getVectorSchemaRoot(ArrowReader.java:65)
	at org.apache.spark.sql.execution.python.ArrowPythonRunner$$anon$1.read(ArrowPythonRunner.scala:162)
	at org.apache.spark.s